<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/PeopleWearingMaskData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## People wearing mask county level data from NYtimes

* https://github.com/nytimes/covid-19-data/tree/master/mask-use

In [1]:
!pip install wget


  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=94d49695298ac54e2be5fb1c5c6dcd6b81c0995f459017b99be2fddaac4b8eab
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import wget
from pandas import Series, datetime
from pandas.plotting import scatter_matrix, autocorrelation_plot
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
import random
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from xgboost import XGBClassifier
from sklearn.mixture import GaussianMixture

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
filename = wget.download('https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv')
filename

'mask-use-by-county.csv'

In [4]:
mask_data = pd.read_csv(filename,low_memory=False)
print(mask_data.shape)

(3142, 6)


In [5]:
mask_data.head()

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459


## FIPS data

In [6]:
US_population_filename = wget.download('https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv')
county_population_US = pd.read_csv(US_population_filename,low_memory=False)
print(county_population_US.shape)

county_population_US.head(2)

(3195, 4)


,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869


In [7]:
county_population_US['COUNTYFP'] = county_population_US['countyFIPS']

## Amalgamation

In [8]:
mask_state_data = pd.merge(mask_data,county_population_US,how='inner' ,on=['COUNTYFP'])
mask_state_data.shape

(3142, 10)

In [9]:
mask_state_data.head(2)

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,countyFIPS,County Name,State,population
0,1001,0.053,0.074,0.134,0.295,0.444,1001,Autauga County,AL,55869
1,1003,0.083,0.059,0.098,0.323,0.436,1003,Baldwin County,AL,223234


## Data processing

In [11]:
ca_mask_data = mask_state_data[mask_state_data['State'] == 'CA']
ca_mask_data.head()

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,countyFIPS,County Name,State,population
186,6001,0.019,0.008,0.055,0.123,0.795,6001,Alameda County,CA,1671329
187,6003,0.025,0.085,0.088,0.190,0.612,6003,Alpine County,CA,1129
188,6005,0.045,0.013,0.099,0.188,0.655,6005,Amador County,CA,39752
189,6007,0.015,0.043,0.111,0.204,0.626,6007,Butte County,CA,219186
190,6009,0.045,0.019,0.098,0.276,0.562,6009,Calaveras County,CA,45905
